In [1]:
import os, sys, email,re
import numpy as np 
import pandas as pd
# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set_style('whitegrid')
import wordcloud

# Network analysis
import networkx as nx

# NLP
from nltk.tokenize.regexp import RegexpTokenizer

from subprocess import check_output

from sklearn.feature_extraction.text import TfidfVectorizer


import gensim
from gensim import corpora
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords 
import string

In [2]:
df = pd.read_csv('df_cleaned.csv')
df['body'] = df['body'].astype(str)
df['clean_content'] = df['body'].apply(lambda x: ''.join([char.lower() for char in x if char not in string.punctuation and not char.isdigit()]))
mask = df['clean_content'].str.contains('sell enron stock', na=False)
df.head()

,subject,sender,recipients,body,clean_content
0,Re: Newsgroups,billc@greenbuilder.com,strawbale@crest.org,What other cool newsgroups are available for u...,what other cool newsgroups are available for u...
1,assoc. for west desk,phillip.allen@enron.com,celeste.roberts@enron.com,"Celeste, I need two assoc.analyst for the west...",celeste i need two assocanalyst for the west g...
2,Re:,phillip.allen@enron.com,william.kelly@enron.com,"Will, I didn't get to review this. I will give...",will i didnt get to review this i will give yo...
3,Re: Electric Overage (1824.62),phillip.allen@enron.com,stagecoachmama@hotmail.com,"Lucy, I will call you this afternoon to discus...",lucy i will call you this afternoon to discus ...
4,Daily Duties,phillip.allen@enron.com,pallen70@hotmail.com,Forwarded by Phillip K AllenHOUECT on 0820200...,forwarded by phillip k allenhouect on pm l...


In [3]:
# Select the data from df using the mask
df[mask]
# Create a list of terms to search for
searchfor = ['enron stock', 'sell stock', 'stock bonus', 'sell enron stock']

# Filter cleaned emails on searchfor list and select from df 
filtered_emails = df[df.clean_content.str.contains('|'.join(searchfor), na=False)]
filtered_emails.head()

,subject,sender,recipients,body,clean_content
288,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,phillip.allen@enron.com,kim.bolton@enron.com,Thanks for the information. It would be helpfu...,thanks for the information it would be helpful...
583,RE: howdy!!,al.pollard@newpower.com,k..allen@enron.com,"Philip, What is going on down the street? Curi...",philip what is going on down the street curiou...
701,Update to Merger Q&A,resources.human@enron.com,dl-ga-all_enron_worldwide2@enron.com,We've updated the Merger QA document on our En...,weve updated the merger qa document on our enr...
1710,Re: ? about turf,john.arnold@enron.com,clayton.vernon@enron.com,"As long a I own Enron stock, the desk are my c...",as long a i own enron stock the desk are my co...
1941,Enron Mentions,m..schmidt@enron.com,(No Recipients),STOCKWATCH Enron higher after Merrill Lynch's ...,stockwatch enron higher after merrill lynchs c...


In [4]:
# Create flag variable where the emails match the searchfor terms
df['flag'] = np.where((df['clean_content'].str.contains('|'.join(searchfor)) == True), 1, 0)

# Count the values of the flag variable
count = df['flag'].value_counts()
print(count)

flag
0    229563
1      1637
Name: count, dtype: int64


In [5]:
# Tokenization
df['clean_content'] = df['clean_content'].apply(lambda row: word_tokenize(row) if isinstance(row, str) else row)

# Remove whitespace, convert to lowercase, and replace non-alphabetic characters
df['clean_content'] = df['clean_content'].apply(lambda row: [word.lower() for word in row if word.isalpha()] if isinstance(row, list) else row)

# Remove all stopwords and punctuation
stop_words = set(stopwords.words('english'))
df['clean_content'] = df['clean_content'].apply(lambda row: [word for word in row if word not in stop_words] if isinstance(row, list) else row)

# Join the cleaned tokens back to a string (if needed)
df['clean_content'] = df['clean_content'].apply(lambda row: ' '.join(row) if isinstance(row, list) else row)

# Display the first few rows to verify
df.head()

,subject,sender,recipients,body,clean_content,flag
0,Re: Newsgroups,billc@greenbuilder.com,strawbale@crest.org,What other cool newsgroups are available for u...,cool newsgroups available u alternative thinke...,0
1,assoc. for west desk,phillip.allen@enron.com,celeste.roberts@enron.com,"Celeste, I need two assoc.analyst for the west...",celeste need two assocanalyst west gas trading...,0
2,Re:,phillip.allen@enron.com,william.kelly@enron.com,"Will, I didn't get to review this. I will give...",didnt get review give feedback tomorrow mornin...,0
3,Re: Electric Overage (1824.62),phillip.allen@enron.com,stagecoachmama@hotmail.com,"Lucy, I will call you this afternoon to discus...",lucy call afternoon discus thing email phillip,0
4,Daily Duties,phillip.allen@enron.com,pallen70@hotmail.com,Forwarded by Phillip K AllenHOUECT on 0820200...,forwarded phillip k allenhouect pm lucy gonzal...,0


In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

# Lemmatize words
lemma = WordNetLemmatizer()
normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())

# Stem words
porter= PorterStemmer()
cleaned_text = " ".join(porter.stem(token) for token in normalized.split())
print (cleaned_text)

df.head()

NameError: name 'punc_free' is not defined

In [7]:
# Define stopwords to exclude
stop = set(stopwords.words('english'))
stop.update(("to", "cc", "subject", "http", "from", "sent", "ect", "u", "fwd", "www", "com", 'html'))

# Define punctuations to exclude and lemmatizer
exclude = set(string.punctuation)

In [8]:
# Import the lemmatizer from nltk
lemma = WordNetLemmatizer()

def clean(text, stop):
    text = str(text).rstrip()
    stop_free = " ".join([i for i in text.lower().split() if((i not in stop) and (not i.isdigit()))])
    punc_free = ''.join(i for i in stop_free if i not in exclude)
    normalized = " ".join(lemma.lemmatize(i) for i in punc_free.split())      
    return normalized

In [9]:
# Clean the emails in df and print results
text_clean=[]
for text in df['clean_content']:
    text_clean.append(clean(text, stop).split())    

In [10]:
text_clean[0][:10]

['cool',
 'newsgroups',
 'available',
 'alternative',
 'thinker',
 'rammed',
 'earth',
 'cob',
 'etc',
 'list']

In [11]:
# Define the dictionary
dictionary = corpora.Dictionary(text_clean)
dictionary.filter_extremes(no_below=5, keep_n=50000)
# Define the corpus 
corpus = [dictionary.doc2bow(text) for text in text_clean]

In [12]:
print(dictionary)

Dictionary<50000 unique tokens: ['alternative', 'available', 'bc', 'bill', 'bldg']...>


In [13]:
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 2),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [14]:
# Define the LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=5)

# Save the topics and top 5 words
topics = ldamodel.print_topics(num_words=5)

# Print the results
for topic in topics:
    print(topic)

(0, '0.012*"please" + 0.009*"email" + 0.008*"may" + 0.007*"agreement" + 0.007*"deal"')
(1, '0.018*"pm" + 0.017*"original" + 0.016*"message" + 0.013*"thanks" + 0.011*"know"')
(2, '0.014*"image" + 0.010*"email" + 0.007*"day" + 0.006*"get" + 0.006*"click"')
(3, '0.011*"power" + 0.010*"energy" + 0.008*"company" + 0.007*"enron" + 0.007*"ha"')
(4, '0.027*"td" + 0.026*"width" + 0.023*"tr" + 0.018*"br" + 0.017*"table"')


In [ ]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)

In [ ]:
pyLDAvis.display(lda_display)

In [ ]:
dominant_topics = []

# Iterate through each document in the corpus
for i, row in enumerate(ldamodel[corpus]):
    # Sort topics by the highest contribution
    row = sorted(row, key=lambda x: x[1], reverse=True)
    # Append the dominant topic number to the list
    dominant_topics.append(row[0][0])

# Assuming `df` is your original DataFrame
# Add a new column 'Dominant_Topic' to the DataFrame
df['Dominant_Topic'] = dominant_topics

# Display the updated DataFrame
df.head()

In [ ]:
def get_topic_details(ldamodel, corpus):
    # Initialize an empty list to store rows
    rows_list = []

    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if i == 0:  # dominant topic
                wp = ldamodel.show_topic(topic_num)
                # Append to the list instead of appending to DataFrame directly
                rows_list.append([topic_num, prop_topic])

    # Create a DataFrame from the collected rows
    topic_details_df = pd.DataFrame(rows_list, columns=['Dominant_Topic', '% Score'])

    return topic_details_df

# Assuming you have `ldamodel` and `corpus` defined, you can call the function:
topic_details_df = get_topic_details(ldamodel, corpus)

# Display the head of the DataFrame
topic_details_df.head()

In [ ]:
# Flag all rows where Dominant_Topic is 1
df['flag'] = np.where(df['Dominant_Topic'] == 1, 1, 0)


df[df['flag'] == 1]
